In [1]:
from typing import List, Dict, Set, Any, Optional, Tuple, Literal, Callable
import numpy as np
import torch
from torch import Tensor
import sigkernel
import os
import sys
import tslearn
import tslearn.metrics
import ksig

from kernels.abstract_base import TimeSeriesKernel, StaticKernel
from kernels.static_kernels import LinearKernel, RBFKernel, PolyKernel
from kernels.integral_type import StaticIntegralKernel
from kernels.sig_pde import SigPDEKernel
from kernels.sig_trunc import TruncSigKernel
from kernels.gak import GlobalAlignmentKernel, sigma_gak

In [6]:
import ksig
import timeit

#### Test GAK ####
N=20
N2=10
T, d = 40, 8
torch.manual_seed(0)
X = torch.randn(N, T, d, dtype=torch.float64).to("cuda") / d
Y = torch.randn(N2, T, d, dtype=torch.float64).to("cuda") / d
X_np = X.cpu().numpy()
Y_np = Y.cpu().numpy()

sigma = tslearn.metrics.sigma_gak(X_np)
gak = tslearn.metrics.cdist_gak
ksigker = ksig.kernels.GlobalAlignmentKernel(static_kernel=ksig.static.kernels.RBFKernel(bandwidth=sigma))
mine = GlobalAlignmentKernel(RBFKernel(sigma=sigma), normalize=True)
# ksigker = ksig.kernels.SignatureKernel(static_kernel=ksig.static.kernels.RBFKernel(bandwidth=sigma), n_levels=5, order=1)
# mine = TruncSigKernel(RBFKernel(sigma=sigma), normalize=True, trunc_level=5, geo_order=1)


#out = gak(X, X, sigma=sigma)
out2 = ksigker(X_np, X_np)
out3 = mine(X, X)
#print(out)
print(out2)
print(out3)
print(np.mean(np.abs(out2 - out3.cpu().numpy())))

def function1():
    gak(X, Y, sigma=sigma)

def function2():
    ksigker(X_np, Y_np)

def function3():
    with torch.no_grad():
        mine(X, Y)

# # Measure the execution time of function 1
# execution_time1 = timeit.timeit(function1, number=1)

# Measure the execution time of function 2
execution_time2 = timeit.timeit(function2, number=10)

# Measure the execution time of function 3
execution_time3 = timeit.timeit(function3, number=10)

#print("Execution time of function 1:", execution_time1)
print("Execution time of function 2:", execution_time2)
print("Execution time of function 3:", execution_time3)

[[1.         0.7540992  0.70017813 0.73076815 0.79172403 0.74061951
  0.77944886 0.75342208 0.73762893 0.75941637 0.75821997 0.7475975
  0.70071841 0.73777793 0.74828175 0.7118317  0.70268679 0.78227631
  0.78041343 0.79369403]
 [0.7540992  1.         0.69360242 0.71461977 0.7366674  0.64711318
  0.75684254 0.73124829 0.73688774 0.6910495  0.7338578  0.67236507
  0.69116014 0.67785651 0.67920512 0.6837755  0.69379764 0.68345998
  0.72759608 0.69094143]
 [0.70017813 0.69360242 1.         0.71696036 0.72487987 0.73577635
  0.68342452 0.72977964 0.75192665 0.68520892 0.68761219 0.67551447
  0.71254332 0.68042519 0.67846466 0.68424072 0.66775809 0.67667531
  0.70367363 0.72839963]
 [0.73076815 0.71461977 0.71696036 1.         0.76533235 0.77556055
  0.7385337  0.76252214 0.77793273 0.74026728 0.73721612 0.75681035
  0.75333759 0.71347713 0.72466037 0.75355237 0.78362487 0.73385204
  0.74189952 0.73877157]
 [0.79172403 0.7366674  0.72487987 0.76533235 1.         0.72972139
  0.75248879 0.71

In [ ]:
#### test dimensions of TimeSeriesKernels ####

N=3
N2=4
T, d = 40, 5
X = torch.randn(N, T, d, dtype=torch.float64) / d
Y = torch.randn(N2, T, d, dtype=torch.float64) / d

inputs = [
    (X, X),
    (X, Y),
    (X[0], X[0]),
    (X[0], Y[0]),
    (X[0], Y),
    (X, Y[0]),
]
diag_inputs = [
    (X, X),
    (X, Y[:N]),
    (X[0], X[0]),
    (X[0], Y[0]),
]
def test_kernel(ker: TimeSeriesKernel, inputs, diag=False):
    print(ker)
    for X, Y in inputs:
        out = ker(X, Y, diag, normalize=False)
        out_normalize = ker(X, Y, diag, normalize=True)
        print(out)
        print(out_normalize)
        print(out.shape)
    print()


sigker = TruncSigKernel(static_kernel=RBFKernel(), 
                        trunc_level=4, 
                        geo_order=1,
                        only_last=True,)
test_kernel(sigker, inputs)
test_kernel(sigker, diag_inputs, True)

sigpde = SigPDEKernel(static_kernel=RBFKernel(),
                     dyadic_order=3,)
test_kernel(sigpde, inputs)
test_kernel(sigpde, diag_inputs, True)

intker = StaticIntegralKernel(static_kernel=RBFKernel())
test_kernel(intker, inputs)
test_kernel(intker, diag_inputs, True)

In [ ]:
import numpy as np
import ksig

# Number of signature levels to use.
n_levels = 5 

# Use the RBF kernel for vector-valued data as static (base) kernel.
static_kernel = ksig.static.kernels.RBFKernel() 

# Instantiate the signature kernel, which takes as input the static kernel.
n_levels = 5
order = 1
sig_kernel = ksig.kernels.SignatureKernel(n_levels=n_levels, order=order, static_kernel=static_kernel)

# Generate 10 sequences of length 50 with 5 channels.
n_seq, l_seq, n_feat = 10, 50, 5 
X = np.random.randn(n_seq, l_seq, n_feat)

# Sequence kernels take as input an array of sequences of ndim == 3,
# and work as a callable for computing the kernel matrix. 
K_XX = sig_kernel(X)  # K_XX has shape (10, 10).

# The diagonal kernel entries can also be computed.
K_X = sig_kernel(X, diag=True)  # K_X has shape (10,).

# Generate another array of 8 sequences of length 20 and 5 features.
n_seq2, l_seq2 = 8, 20
Y = np.random.randn(n_seq2, l_seq2, n_feat)

# Compute the kernel matrix between arrays X and Y.
K_XY = sig_kernel(X, Y)  # K_XY has shape (10, 8)
K_XY